# Beam with prescribed displacement $d$, using Lagrange multipliers 

**Summary**: we consider a beam that is attached to a base on the bottom, and then prescribe a displacement to the top of the beam. For a small displacement, the beam will compress but stay straight, but when the displacement reaches a critical value, the system becomes unstable and the beam will buckle, which will make it bend to either the left side or right side; either option is equally likely.

**Description of the system**: The beam consists of $n$ compressible beam elements, of which element $i$ has length $L_i$ and is compressible with a stiffness $C_i$. There are torsion springs between the base and the first element and between subsequent elements; these torsion springs have spring constant $K_i$. The beam also has an unspecified cross sectional area $A_i$, but this is absorbed into the stiffness $C_i$.

When the beam deforms, the amount element $i$ is compressed is described by the strain $\epsilon_i$. We use Hencky strains, which are defined as $\epsilon_i = \ln\left(\frac{L_i'}{L}\right)$, where $L_i$ is the reference length of the beam element and $L_i'$ the length of the deformed beam element.
The angle the elements make with the vertical are $q_i$. In the undeformed reference configuration the beam is oriented straight up, and therefore these angles are initially all zero.
The displacement prescribed to the top of the beam in downward direction is $d$. It takes a certain unknown force $P$ to apply this displacement.

**Problem**:
* Input: displacement $d$ (applied in increments)
* Parameters: lengths $L_i$, spring constants $K_i$, stiffnesses $C_i$.
* Output: angles $q_i$, strains $\epsilon_i$, optionally force $P$. This output can also be described by the final positions of nodes location at the top and bottom of the beam as well as the joints between elements.

The beam can be represented by a path graph, where nodes are located at the top and bottom of the beam as well as the joints between elements, and beam elements are the edges between these nodes.
Because the displacement $d$ is applied in increments, such that the top of the beam is gradually pushed downward, this results in sequential graph data.

![title](beamdiagram.png)

# Sample buckling beams
* Pick n between 2 and 10
* Pick $K_i$ from range $[0.25, 4.0] = [2^{-2}, 2^2]$ (log-uniform distribution)
* Pick $C_i$ from range $[8.0, 128] = [2^{3}, 2^7]$ (log-uniform distribution)
* Pick $L_i$ from range $[0.25, 4.0] = [2^{-2}, 2^2]$ (log-uniform distribution)


## n segments, assuming all $\alpha_i = 0$, $C_i$, $L_i$, $K_i$ vary per segment
Strain energy:
$$U(\mathbf{q}, \epsilon) = \frac{1}{2}\left[K_1q_1^2 + \sum_{i=2}^n K_i(q_i-q_{i-1})^2+ \sum_{i=1}^n L_i C_i \epsilon_i^2\right] $$

Applied displacement:
$$d = \sum_{i=1}^n L_i \left[1-\exp(\epsilon_i)\cos q_i\right]$$
Constraint of applied displacement:
$$g(\mathbf{q}, \mathbf{\epsilon}) = d - \sum_{i=1}^n L_i \left[1-\exp(\epsilon_i)\cos q_i\right] = 0$$


## Lagrange multipliers
Find angles $q_i$ and strains $\epsilon_i$ that minimize strain energy $U(\mathbf{q}, \epsilon)$ subject to constraint $g(\mathbf{q}, \mathbf{\epsilon}) = d - \sum_{i=1}^n L_i \left[1-\exp(\epsilon_i)\cos q_i\right] = 0$ with $d$ the applied displacement at the tip. 

Lagrangian function:
$$\mathcal{L}(\mathbf{q}, \epsilon, \lambda) = U(\mathbf{q}, \epsilon) + \lambda g(\mathbf{q}, \epsilon)$$
$$ = \frac{1}{2}\left[K_1q_1^2 + \sum_{i=2}^n K_i(q_i-q_{i-1})^2 + \sum_{i=1}^n L_i C_i \epsilon_i^2\right] + \lambda \left\{d - \sum_{i=1}^n L_i \left[1-\exp(\epsilon_i)\cos q_i\right] \right\}$$


## Derivatives:
$$\frac{\partial\mathcal{L}}{\partial q_i} =  0$$
$$\frac{\partial\mathcal{L}}{\partial \epsilon_i} = 0$$
$$\frac{\partial\mathcal{L}}{\partial \lambda} = 0$$
meaning, $2n+1$ equations, $2n+1$ unknowns (because $\lambda$ now counts too)

Expressions for the derivatives:
$$\frac{\partial\mathcal{L}}{\partial q_i} = \begin{cases}
(K_1+K_2)q_1 - K_2q_2 - \lambda L_1\exp(\epsilon_i)\sin(q_1),  \quad & i=1,\\
(K_{i} + K_{i+1})q_i - K_{i}q_{i-1} - K_{i+1}q_{i+1} - \lambda L_i\exp(\epsilon_i)\sin(q_i),  \quad & 1<i<n\\
K_n(q_n - q_{n-1}) - \lambda L_n\exp(\epsilon_i)\sin(q_n),  \quad & i=n
\end{cases},$$
$$\frac{\partial\mathcal{L}}{\partial \epsilon_i} = L_i C_i \epsilon_i + \lambda L_i \exp(\epsilon_i) \cos(q_i)$$
$$\frac{\partial\mathcal{L}}{\partial \lambda} = g(\mathbf{q}, \epsilon) = d - \sum_{i=1}^n L_i \left[1-\exp(\epsilon_i)\cos q_i\right]$$


## Second derivatives:
$$\frac{\partial ^2 \mathcal{L}(\mathbf{q},\epsilon)}{\partial q_i\partial q_j}
= \begin{cases}
K_i+K_{i+1}- \lambda L_i\exp(\epsilon_i)\cos(q_i), \quad & i=j, i<n\\
K_n- \lambda L_i\exp(\epsilon_i)\cos(q_i), \quad & i=j=n\\
-K_i, \quad & j=i-1\\
-K_j, \quad & j=i+1\\
0, \quad & \textrm{else}
\end{cases}$$

$$\frac{\partial ^2 \mathcal{L}(\mathbf{q}, \epsilon)}{\partial \epsilon_i\partial \epsilon_j} = 
\begin{cases}L_iC_i + \lambda L_i \exp(\epsilon_i)\cos(q_i), \quad & i=j \\
0 \quad & \textrm{else}
\end{cases}$$

$$\frac{\partial ^2 \mathcal{L}(\mathbf{q}, \epsilon)}{\partial q_i \partial \epsilon_j} = 
\begin{cases}-\lambda L_i \exp(\epsilon_i) \sin q_i, \quad & i=j \\
0 \quad & \textrm{else}
\end{cases}$$

$$\frac{\partial ^2 \mathcal{L}(\mathbf{q}, \epsilon)}{\partial \lambda \partial q_i} = -L_i \exp(\epsilon_i) \sin q_i$$
$$\frac{\partial ^2 \mathcal{L}(\mathbf{q}, \epsilon)}{\partial \lambda \partial \epsilon_i} = L_i \exp(\epsilon_i) \cos q_i$$
$$\frac{\partial ^2 \mathcal{L}(\mathbf{q}, \epsilon)}{\partial \lambda^2} = 0$$


# Start code
## Import/define things

In [ ]:
import matplotlib.pyplot as plt
import scipy.optimize as scopt
import numpy as np
from scipy.sparse.linalg import eigsh
import os
import sklearn.neighbors as skln
import scipy.sparse as sps
import warnings

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r'\usepackage{{amsmath}}',
    'font.size': 12
})


In [ ]:
def angles_to_xy(angles, L=None):
    n = len(angles)
    if L is None:
        L = np.ones(n)

    x = np.zeros(n + 1)
    y = np.zeros(n + 1)
    x[1:] = np.cumsum(L*np.sin(angles))
    y[1:] = np.cumsum(L*np.cos(angles))
    return x, y

In [ ]:
def plot_beam(angles, ax=None, K=None, L=None, **kwargs):
    n = len(angles)
    if K is None:
        K = np.ones(n+1)
    else:
        K = np.concatenate((K, [0]))
    if L is None:
        tl = n  # total length
    else:
        tl = np.sum(L)  # total length

    return_fig = False
    if ax is None:
        return_fig = True
        fig, ax = plt.subplots()
    x, y = angles_to_xy(angles, L=L)
    ax.plot(x, y, **kwargs)
    kwargs['color'] = 'gray'
    if 'c' in kwargs:
        del kwargs['c']
    ax.scatter(x, y, s=K, **kwargs)
    # create rectangle patch
    margin = 0.2
    factor = margin + 1.0
    ax.add_patch(plt.Rectangle((-tl*factor, -tl*factor), 2*tl*factor, tl*factor, facecolor='gray', linewidth=0))
    ax.set_xlim(-tl*factor, tl*factor)
    ax.set_ylim(-tl*factor, tl*factor)
    ax.set_aspect('equal')

    if return_fig:
        return fig

def plot_add_beam(angles, ax, c=None,  K=None, L=None, **kwargs):
    n = len(angles)
    if K is None:
        K = np.ones(n+1)
    else:
        K = np.concatenate((K, [0]))
    x, y = angles_to_xy(angles, L)
    # if strains is not None:
    #     kwargs['c'] = strains
    if c is not None:
        if 'c' in kwargs:
            del kwargs['c']
        for i in range(len(x) - 1):
            ax.plot(x[i:i+2], y[i:i+2], color=plt.cm.coolwarm(c[i]), **kwargs)
    else:
        ax.plot(x, y, **kwargs)

    # plot nodes (rotational springs)
    kwargs['color'] = 'black'
    if 'c' in kwargs:
        del kwargs['c']
    ax.scatter(x, y, s=K, **kwargs)
    return ax

## Define derivatives of Lagrangian

In [ ]:
def dLdq(x,K,C,L,d):
    # x[1:n+1] are the angles, x[n+1:] are the strains, x[0] is the force (lambda)
    # therefore len(K)=len(C)=len(L)=n=(len(x)-1)/2
    lamb = x[0]
    n = (len(x)-1)//2
    q = np.asanyarray(x)[1:n+1]
    e = np.asanyarray(x)[n+1:]
    K = np.asanyarray(K)
    C = np.asanyarray(C)
    L = np.asanyarray(L)
    temp = np.empty(len(x), dtype=float)

    # print('q:', q)
    # print('e:', e)
    # print('K:', K)
    # print('C:', C)
    # print('L:', L)

    # dL/dλ
    temp[0] = d - np.sum(L*(1-np.exp(e)*np.cos(q)))
    # dL/dq, case i = 0
    temp[1] = (K[0] + K[1])*q[0] - K[1]*q[1]
    # dL/dq, case i = 1 to n-2
    temp[2:n] = (K[1:-1] + K[2:])*q[1:-1] - K[1:-1]*q[:-2] - K[2:]*q[2:]
    # dL/dq, case i = n - 1
    temp[n] = K[-1]*(q[-1] - q[-2])
    temp[1:n+1] -= lamb*L*np.exp(e)*np.sin(q)
    # dL/de
    temp[n+1:] = L*C*e + lamb*L*np.exp(e)*np.cos(q)
    return temp

def d2Ldq2(x,K,C,L,d):
    # x[1:n+1] are the angles, x[n+1:] are the strains, x[0] is the force (lambda)
    lamb = x[0]
    n = (len(x)-1)//2
    q = np.asanyarray(x)[1:n+1]
    e = np.asanyarray(x)[n+1:]
    K = np.asanyarray(K)
    C = np.asanyarray(C)
    L = np.asanyarray(L)
    temp = np.zeros((len(x), len(x)), dtype=float)

    # λ, λ  # d2L/dλdλ = 0

    # λ, q  # d2L/dλdq
    temp[0, 1:n+1] = - L*np.exp(e)*np.sin(q)
    temp[1:n+1, 0] = temp[0, 1:n+1]

    # λ, e  # d2L/dλde
    temp[0, n+1:] = L*np.exp(e)*np.cos(q)
    temp[n+1:, 0] = temp[0, n+1:]

    # q, q  # d2L/dqdq, diagonal
    np.fill_diagonal(temp[1:n, 1:n], K[:-1] + K[1:] - L[:-1]*lamb*np.exp(e[:-1])*np.cos(q[:-1]))
    # d2L/dqdq, off-diagonal
    np.fill_diagonal(temp[2:n+1, 1:n+1], -K[1:])
    np.fill_diagonal(temp[1:n+1, 2:n+1], -K[1:])
    # d2L/dqdq, final element
    temp[n, n] = K[-1] - L[-1]*np.exp(e[-1])*lamb*np.cos(q[-1])

    # e, q  # d2L/dedq
    np.fill_diagonal(temp[n+1:, 1:n+1], -lamb*L*np.exp(e)*np.sin(q))
    np.fill_diagonal(temp[1:n+1, n+1:], -lamb*L*np.exp(e)*np.sin(q))

    # e, e  # d2L/dede
    np.fill_diagonal(temp[n+1:, n+1:], L*C + lamb*L*np.exp(e)*np.cos(q))

    return temp

def d2Udq2(x,K,C,L,d):
    # x[1:n+1] are the angles, x[n+1:] are the strains, x[0] is the force (lambda)
    lamb = x[0]
    n = (len(x)-1)//2
    q = np.asanyarray(x)[1:n+1]
    e = np.asanyarray(x)[n+1:]
    K = np.asanyarray(K)
    C = np.asanyarray(C)
    L = np.asanyarray(L)
    temp = np.zeros((len(x)-1, len(x)-1), dtype=float)

    # q, q  # d2L/dqdq, diagonal
    np.fill_diagonal(temp[0:n-1, 0:n-1], K[:-1] + K[1:])
    # d2L/dqdq, off-diagonal
    np.fill_diagonal(temp[1:n, 0:n], -K[1:])
    np.fill_diagonal(temp[0:n, 1:n], -K[1:])
    # d2L/dqdq, final element
    temp[n-1, n-1] = K[-1]

    # e, e  # d2L/dede
    np.fill_diagonal(temp[n:, n:], L*C)

    return temp


In [ ]:
def displacement(angles, strains, L):
    return np.sum(L*(1-np.exp(strains)*np.cos(angles)))

In [ ]:
def is_stable(K):
    # Test 1: minors
    stable1 = True
    for i in range(3, len(K)+1):
        # print(K[:i,:i].shape)
        # print(np.linalg.det(K[:i,:i]))
        if np.linalg.det(K[:i,:i]) > 0:
            stable1 = False
            break

    # Test 2: eigenvalues
    # test if there is exactly one negative eigenvalue
    stable2 = np.sum(np.linalg.eigvals(K) < 0) == 1

    if stable1 and stable2:
        return True
    elif not stable1 and not stable2:
        return False
    else:
        print('Minors:\n', [np.linalg.det(K[:i,:i]) for i in range(3, len(K)+1)])
        print('Eigenvalues:\n', np.linalg.eigvals(K))
        warnings.warn('Stability tests are inconsistent')
        return stable2

In [ ]:
def get_eigenmodes(K, K_orig, k=5):  #, threshold=0.1, k=5):
    N = len(K_orig)
    # K: second derivatives of the Lagrangian
    # K_orig: second derivatives of the energy
    # returns matrix where the columns are the eigenmodes

    C = K[[0], 1:]  # constraint matrix: dg/dq = d2L/dλdq
    Q, R = np.linalg.qr(C.T, mode='complete')
    Q2 = Q[:, 1:]

    asdf = np.dot(Q2.T, np.dot(K_orig, Q2))
    eigenvalues, eigenvectors = eigsh(asdf, k=k, which='SA')
    # inds = np.where(eigenvalues < threshold)[0]
    # eigenvectors = eigenvectors[:, inds]
    eigenmodes = np.dot(Q2, eigenvectors)
    return eigenmodes

In [ ]:
def is_posdef(K):
    """Check if a symmetric matrix positive definite. ONLY WORKS FOR SYMMETRIC MATRICES (or hermitian I guess?)

    Parameters
    ----------
    K : array-like
        Array to test positive definiteness of

    Returns
    -------
    bool
        True if K is positive definite, False otherwise
    """
    try:
        np.linalg.cholesky(K)
        return True
    except np.linalg.linalg.LinAlgError as err:
        if 'Matrix is not positive definite' in repr(err):
            return False
        else:
            raise

## Define solve function

In [ ]:
def sol_valid(q_sol_temp, KK, L, d_temp, verbose=False):
    N = len(L)
    if np.isnan(q_sol_temp).any():
        if verbose:
            print(q_sol_temp)
            print(f'encountered np.nan in solution after perturbation')
        return False
    # # check stability
    elif not is_stable(KK):
        if verbose:
            print(f'q_sol={q_sol_temp} is unstable\n')
        return False
    # check if solution indeed has displacement d
    elif np.abs(displacement(q_sol_temp[1:N+1], q_sol_temp[N+1:], L) - d_temp) > 1e-6:
        if verbose:
            print(f'q_sol={q_sol_temp} does not have displacement d={d_temp} (d={displacement(q_sol_temp[1:N+1], q_sol_temp[N+1:], L)})')
        return False
    # check if lambda is not zero
    elif q_sol_temp[0] <= 1e-7 and d_temp != 0:
        return False
    # make sure not all strains are zero
    elif np.all(np.abs(q_sol_temp[N+1:]) < 1e-9) and d_temp != 0:
        return False
    # make sure not all angles are equal
    elif max(q_sol_temp[1:N+1]) - min(q_sol_temp[1:N+1]) < 1e-7 and np.max(q_sol_temp[1:N+1]) > 1e-6:
        return False
    else:
        return True

In [ ]:
def solve_buckling_beam(N, K, C, L, d, verbose=False):
    q_sol = np.empty((len(d), 2*N+1), dtype=float)
    L_tot = np.sum(L)

    # iterate over all values of the displacement d
    for i, d_temp in enumerate(d):
        if i == 0:
            q0 = [0.1] + [0.0]*2*N  # initial guess: all angles and strains are zero, lambda = 0.1
        else:
            q0 = np.copy(q_sol[i-1]) # use solution previous time step as initial guess
            # however, make sure the lambda is not zero (can cause problems)
            if np.abs(q0[0]) < 1e-4:
                q0[0] = 0.1

        sol = scopt.root(dLdq, q0,
                                args=(K, C, L, d_temp),
                                # jac=d2Ldq2
                                )
        q_sol_temp = sol.x

        KK = d2Ldq2(q_sol_temp, K, C, L, d_temp)

        if sol_valid(q_sol_temp, KK, L, d_temp, verbose=verbose):
            q_sol[i] = q_sol_temp

        elif not is_stable(KK):  # if the solution is unstable, try perturbing the initial guess
            K_orig = d2Udq2(q_sol_temp, K, C, L, d_temp)
            mode = get_eigenmodes(KK, K_orig, k=min(1, N)).T[0]

            magn = 0.1  # perturbation magnitude
            max_pert = np.max(np.abs(mode))

            if verbose:
                print(f'eigenvector={mode}')

            perturbation = magn*mode/max_pert
            q0_temp2 = np.copy(q0)
            q0_temp2[1:] += perturbation
            q_sol_temp = scopt.root(dLdq, q0_temp2,
                                    args=(K, C, L, d_temp),
                                    jac=d2Ldq2
                                    ).x

            KK = d2Ldq2(q_sol_temp, K, C, L, d_temp)

            if sol_valid(q_sol_temp, KK, L, d_temp, verbose=True):
                q_sol[i] = q_sol_temp
            else:
                # try again with different magn
                print('perturbed solution is not valid, trying again with different magnitude')
                K_orig = d2Udq2(q_sol_temp, K, C, L, d_temp)
                mode = get_eigenmodes(KK, K_orig, k=min(1, N)).T[0]

                magn = 0.5  # perturbation magnitude
                max_pert = np.max(np.abs(mode))

                if verbose:
                    print(f'eigenvector={mode}')

                perturbation = magn*mode/max_pert
                q0_temp2 = np.copy(q0)
                q0_temp2[1:] += perturbation
                q_sol_temp = scopt.root(dLdq, q0_temp2,
                                        args=(K, C, L, d_temp),
                                        jac=d2Ldq2
                                        ).x

                KK = d2Ldq2(q_sol_temp, K, C, L, d_temp)

                if sol_valid(q_sol_temp, KK, L, d_temp, verbose=True):
                    q_sol[i] = q_sol_temp
                else:
                    raise ValueError('Perturbed solution is invalid')

        else: # if the solution was invalid for another reason than instability
            raise ValueError('Solution is invalid')

    # mirror the result
    temp2 = np.copy(q_sol)
    temp2[:, 1:N+1] = -temp2[:, 1:N+1]  # flip signs of angles
    q_sol = np.stack((q_sol, temp2), axis=1)

    return q_sol

In [ ]:
def new_path(path, n=2, always_number=True):
    '''Create a new path for a file
    with a filename that does not exist yet,
    by appending a number to path (before the extension) with at least n digits.
    (Higher n adds extra leading zeros). If always_number=True,
    the filename will always have a number appended, even if the
    bare filename does not exist either.'''

    # initial path is path + name + n zeros + extension if always_number=True
    # and just path + filename if always_number=False
    name, ext = os.path.splitext(path)
    if always_number:
        savepath = f'{name}_{str(0).zfill(n)}{ext}'
    else:
        savepath = path

    # If filename already exists, add number to it
    i = 1
    while os.path.exists(savepath):
        savepath = f'{name}_{str(i).zfill(n)}{ext}'
        i += 1
    return savepath

## Define plotting functions

In [ ]:
def arr1D_to_str(arr):
    return '[' + ', '.join([f'{elem:.3f}' for elem in arr]) + ']'

In [ ]:
def plot_q_i_vs_d(q_sol, d, K, C, L):
    fig = plt.figure(figsize=(8, 8))
    # create figure with two axes, with shared x-axis
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312, sharex=ax1)
    ax3 = plt.subplot(313, sharex=ax1)

    # q_sol[:, 0] is d, q_sol[:, 1] is lambda, q_sol[:, 2:n+1] are the angles, q_sol[:, n+2:] are the strains
    n = len(C)
    lamb = q_sol[:, 0, 0]
    q = q_sol[:, :, 1:n+1]
    e = q_sol[:, 0, n+1:]

    L_tot = np.sum(L)

    colors = plt.cm.viridis(np.linspace(0, 1, n))
    for j in range(n):  # iterate over beam elements
        for i in [0, 1]:  # iterate over solutions
            # plot q_i
            ax1.scatter(d/L_tot, q[:,i,j], s=2, color=colors[j], label=i*'_' + f'$q_{j}$')
        # plot epsilons
        ax2.scatter(d/L_tot, e[:,j], color=colors[j], s=2, label=f'$\epsilon_{j}$')

    ax1.grid(which='both')
    # fig.ax.set_xlim([60, 90])
    # ticks = np.linspace(-np.pi, np.pi+0.01, 9)
    # ticklabels = [['', 1],  [3, 4], ['', 2],['', 4],
    #             ['', ''], ['', 4], ['', 2], [3, 4], ['', 1]]
    # ticklabels = [f'$\\frac{{{xx}\\pi}}{yy}$' for xx, yy in ticklabels]
    # ticklabels[4] = '0'
    # ticklabels[8] = f'$\\pi$'
    # ticklabels[0] = f'$-\\pi$'
    # ax1.set_yticks(ticks)
    # ax1.set_yticklabels(ticklabels)

    ax1.set_ylabel(f'$q_i$')
    ax1.set_title(f'$K={arr1D_to_str(K)}$,\n$C={arr1D_to_str(C)}$,\n$L={arr1D_to_str(L)}$')
    ax1.legend(loc='upper right')

    ax2.grid(which='both')
    ax2.set_ylabel(f'$\epsilon_i$')
    ax2.legend(loc='upper right')

    lamb[d == 0] = 0.0
    ax3.scatter(d/L_tot, lamb, s=2, color='black', label='$\lambda$')
    ax3.grid(which='both')
    ax3.set_ylabel(f'$\lambda$')
    ax3.set_xlabel('$d/L_{tot}$')
    return fig


In [ ]:
def plot_deformations(q_sol, d, K, C, L):

    N_pl = 9  # number of plots
    N = len(L)  # nr of segments
    Nsqrt = np.ceil(np.sqrt(N_pl)).astype(int)
    fig, axes = plt.subplots(Nsqrt, Nsqrt, figsize=(10, 10))
    inds = np.linspace(0, len(d)-1, N_pl).astype(int)
    L = np.asanyarray(L)

    axes = axes.flatten()

    max_abs_str = max([max([max(np.abs(elem2[N+1:])) for elem2 in elem]) for elem in q_sol if elem != []])

    # iterate over cases to plot
    for ind, ax in zip(inds, axes):

        # plot initial position
        plot_beam([0.0]*N, ax=ax, K=K*10, L=L, color='gray')

        # plot all solutions for this case
        for q_sol_temp in q_sol[ind]:
            angles = q_sol_temp[1:N+1]
            strains = np.array(q_sol_temp[N+1:])
            c = (strains + max_abs_str)/(2*max_abs_str)
            plot_add_beam(angles, ax=ax, c=c, K=K*10, L=L*np.exp(strains))

        ax.set_title(f'd={d[ind]:.2f}')

    # widen margins between plots
    fig.subplots_adjust(top=0.8, wspace=0.2, hspace=0.25)
    fig.suptitle(f'$N={len(C)}$,\n$K={arr1D_to_str(K)}$,\n$C={arr1D_to_str(C)}$,\n$L={arr1D_to_str(L)}$')
    return fig

## Test on a specific case

In [ ]:
N = 2 # number of segments
K = np.ones(N)
C = np.ones(N)
L = np.ones(N)
print(f'N={N}, K={K}, C={C}, L={L}')
n = 50
L_tot = np.sum(L)
d = np.linspace(0, L_tot, n)

# n different values of P
# N segments

q_sol = solve_buckling_beam(N, K, C, L, d, verbose=True)

In [ ]:
# Plot of one load case

N = len(L)  # nr of segments
fig, ax = plt.subplots(figsize=(10, 10))
L = np.asanyarray(L)

# get value of maximum absolute value of the strain (to set colormap limits)
max_abs_str = max([max([max(np.abs(elem2[N+1:])) for elem2 in elem]) for elem in q_sol if elem != []])

# plot initial position
plot_beam([0.0]*N, ax=ax, K=K*100, L=L, color='grey', linewidth=5)

# plot all solutions for this case
ind = 20
for q_sol_temp in q_sol[ind]:
    angles = q_sol_temp[1:N+1]
    strains = np.array(q_sol_temp[N+1:])
    c = (strains + max_abs_str)/(2*max_abs_str)
    plot_add_beam(angles, ax=ax, c=c, K=K*100, L=L*np.exp(strains), linewidth=5)

# ax.set_title(f'd={d[ind]:.2f}')
ax.set_ylim([-0.5, 2.3])
ax.set_xlim([-1.1, 1.1])

# widen margins between plots
fig.subplots_adjust(top=0.8, wspace=0.2, hspace=0.25)
# fig.suptitle(f'$N={len(C)}$,\n$K={arr1D_to_str(K)}$,\n$C={arr1D_to_str(C)}$,\n$L={arr1D_to_str(L)}$')

In [ ]:
fig = plot_q_i_vs_d(q_sol, d, K, C, L)
fig = plot_deformations(q_sol, d, K, C, L)

## Sample lots of random beams, save figures & results

In [ ]:
import os
import pickle
import datetime
save_folder = r'data\bucklingBeam_d_qe_Lagrangian' + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [ ]:
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

rng = np.random.default_rng()
error_cases = []
for _ in range(1000):
    # n different values of d
    # N segments
    N = rng.integers(2, 11) # number of segments
    L = 2**rng.uniform(-1, 1, N)  # 0.5 to 2
    C = 2**rng.uniform(-1, 1, N)
    K = 2**rng.uniform(-1, 1, N)

    # L = 2**rng.uniform(0, 1, N)  # 1 to 2
    # R = 2**rng.uniform(-3, -2, N) #  1/8 to 1/4
    # C = np.pi*R**2/L  # E A / L, using E=1 and A = pi R^2
    # K = np.pi*R**4/L # 4 E I / L, using E=1 and I = pi R^4 / 4

    print(f'N={N}, K={K}, C={C}, L={L}')
    n = 200
    L_tot = np.sum(L)
    max_d = L_tot  #2.5*np.exp(-4.1 - 0.35*N)*L_tot
    d = np.linspace(0, max_d, n)

    # Give this run a unique name
    date_time_string = str(datetime.datetime.now()).replace(' ', '_').replace(':', '-')
    name = f'bucklingBeam_{date_time_string}'
    path = new_path(os.path.join(save_folder, name + '.pkl'), always_number=False)
    path = os.path.splitext(path)[0]

    try:
        # solve
        q_sol = solve_buckling_beam(N, K, C, L, d)

        # Plot angles, strains, lambda vs d
        fig = plot_q_i_vs_d(q_sol, d, K, C, L)
        path2 = new_path(path + '_q_e_lambda_vs_d.png', always_number=False)
        fig.savefig(path2)
        plt.close(fig)
        # Plot beam deformations
        fig = plot_deformations(q_sol, d, K, C, L)
        path2 = new_path(path + '_deformation.png', always_number=False)
        fig.savefig(path2)
        plt.close(fig)

        # save results
        with open(path + '.pkl', 'wb') as f:
            pickle.dump({'L': L, 'K': K, 'C': C, 'd': d, 'N': N, #'R': R,
                        'q_sol': q_sol}, f)
    except ValueError as e:
        print(f'Error in {name}: {e}')
        error_cases.append({'L': L, 'K': K, 'C': C, 'd': d, 'N': N, #'R': R,
                            })


In [ ]:
print(len(error_cases))
print(error_cases)

In [ ]:
with open(os.path.join(save_folder, 'error_cases.pkl'), 'wb') as f:
    pickle.dump(error_cases, f)

# Check results

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial import Polynomial

In [ ]:

results = []

# Specify the directory where the .pkl files are located
directory = r'data\bucklingBeam_d_qe_Lagrangian2025-01-23_13-11-36'

crit = []
all_C = []
all_L = []
n_sol = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.pkl') and not filename.startswith('error'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            result = pickle.load(f)
            N = len(result['L'])
            # find first load case where the angles of the solution are not all zeros
            for i, q in enumerate(result['q_sol']): # iterate over time steps
                if not np.allclose(q[:, 1:N+1], 0):
                    crit.append(result['d'][i])
                    break
            else:
                # if critical point was not reached, append final applied d
                crit.append(np.nan)
            all_C.append(result['C'])
            all_L.append(result['L'])


In [ ]:
N = np.array([len(C) for C in all_C])
crit = np.array(crit)

In [ ]:
# plot critical point vs N
L_tot = np.array([np.sum(L) for L in all_L])
plt.figure()
plt.scatter(N, crit/L_tot)
plt.xlabel('N')
plt.ylabel('d_crit/L_tot')
# plt.yscale('log')

# Create graph data

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch_geometric as tg

In [ ]:
# Specify the directory where the .pkl files are located
directory = r'data\bucklingBeam_d_qe_Lagrangian2025-01-23_13-11-36'

In [ ]:
# print shapes of all arrays in the first file
for filename in os.listdir(directory):
    if filename.endswith('.pkl') and not filename.startswith('error'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            result = pickle.load(f)
            # print overview of contents
            for key, value in result.items():
                if isinstance(value, np.ndarray):
                    print(f'{key:6} {str(value.shape):15} {value.dtype}')
                elif isinstance(value, list):
                    print(f'{key:6} {len(value):15} {type(value[0])}')
                else:
                    print(f'{key:<6} {value:<15} {type(value)}')
    break


Keys in each results file and their shape and type:
```
* L      (N,)                       ndarray of float64
* K      (N,)                       ndarray of float64
* C      (N,)                       ndarray of float64
* d      (n_time_steps,)            ndarray of float64
* N      -                          integer
* R      (N,)                       ndarray of float64
* q_sol  (n_time_steps, 2, 2N+1)    ndarray of float64
```

In [ ]:
%matplotlib qt
results = []

graphs = []
# Iterate over all files in the directory
for i, filename in enumerate(os.listdir(directory)):
    if filename.endswith('.pkl') and not filename.startswith('error'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            result = pickle.load(f)
            N = result['N']

            # node attributes: rotational stiffness K, last node K=0
            K = np.append(result['K'], 0)

            node_attr = torch.tensor(K).unsqueeze(-1)

            # edge index: 0 -> 1, 1 -> 2, ..., N-1 -> N and reverse
            temp = torch.arange(N)
            edge_index = torch.stack((temp, temp+1), dim=0)
            edge_index = torch.cat((edge_index, edge_index.flip(0)), dim=1)

            d = result['d']

            # edge attributes: length L, stiffness under compression C
            L = result['L']
            edge_attr = torch.tensor(np.stack((L, result['C']), axis=1))
            edge_attr = torch.cat((edge_attr, edge_attr), dim=0)

            # node positions
            angles = result['q_sol'][:, :,1:N+1]  # shape (n_times_steps, 2, N)
            strains = result['q_sol'][:, :,N+1:]
            new_lens = L.reshape(1, 1, -1)*np.exp(strains)
            dx = new_lens*np.sin(angles)
            dy = new_lens*np.cos(angles)
            pos = np.zeros((len(d), 2, N+1, 2))  # shape (n_time_steps, n_solutions=2, N+1, n_dims=2)
            pos[:, :, 1:, 0] = np.cumsum(dx, axis=-1)
            pos[:, :, 1:, 1] = np.cumsum(dy, axis=-1)

            # lambda
            lamb = result['q_sol'][:, :, 0]

            # transpose to shape (N+1, n_time_steps, 2, n_solutions)
            pos = np.transpose(pos, (2, 0, 3, 1))
            pos = torch.tensor(pos)
            graphs.append(tg.data.Data(edge_index=edge_index,
                                       node_attr=node_attr,
                                       pos=pos,
                                       edge_attr=edge_attr,
                                       d=torch.tensor([d]).reshape(1,-1,1),
                                       N=torch.tensor([N]).reshape(1,1),
                                       lamb=torch.tensor(lamb).reshape(1,-1, 2, 1)
                                    )
            )
            # print(graphs[-1])

            # plt.figure()
            # for j in range(N+1):
            #     plt.scatter(pos[:, 0, j, 0], pos[:, 0, j, 1], s=1)
            # plt.gca().set_aspect('equal')



In [ ]:
for graph in graphs[:5]:
    print(graph)

In [ ]:
training_frac = 0.7
N_tr = int(training_frac*len(graphs))
graphs_tr = graphs[:N_tr]
graphs_te = graphs[N_tr:]


print('Training data size:', len(graphs_tr))
print('Test data size:', len(graphs_te))
print('Total:', len(graphs))

In [ ]:
with open('../data/BucklingBeams_data.pkl', 'wb') as f:
    pickle.dump({'data_tr': graphs_tr, 'data_te': graphs_te}, f)

# Create animation

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = r'data\bucklingBeam_d_qe_Lagrangian2025-01-23_13-11-36\bucklingBeam_2025-01-23_13-11-50.525268.pkl'
with open(path, 'rb') as f:
    result = pickle.load(f)

In [ ]:
def arr1D_to_str(arr):
    return '[' + ', '.join([f'{elem:.3f}' for elem in arr]) + ']'

In [ ]:
# Multiple subplots, one for each selected time step

L = result['L']
K = result['K']
C = result['C']
d = result['d']
N = result['N']

K = np.append(K, 0)

margin = 0.2
factor = margin + 1.0
tl = np.sum(L)

N_pl = 9
Nsqrt = np.ceil(np.sqrt(N_pl)).astype(int)
fig, axes = plt.subplots(Nsqrt, Nsqrt, figsize=(10, 10))
inds = np.linspace(0, len(d) - 1, N_pl).astype(int)
axes = axes.flatten()

max_abs_str = max([max([max(np.abs(elem2[N+1:])) for elem2 in elem]) for elem in result['q_sol'] if elem != []])

for ind, ax in zip(inds, axes):  # iterate over snapshots to plot
    # plot reference configuration
    # x,y = np.zeros(N+1), np.append(0, np.cumsum(L))
    # ax.plot(x, y, color='gray')
    # ax.scatter(x, y, s=K, color='gray')

    for q_sol_temp in result['q_sol'][ind]: # iterate over solutions
        angles = q_sol_temp[1:N+1]
        strains = np.array(q_sol_temp[N+1:])
        c = (strains + max_abs_str) / (2 * max_abs_str)

        new_L = L * np.exp(strains)
        x = np.zeros(N + 1)
        y = np.zeros(N + 1)
        x[1:] = np.cumsum(new_L * np.sin(angles))
        y[1:] = np.cumsum(new_L * np.cos(angles))
        for i in range(len(x) - 1):
            ax.plot(x[i:i+2], y[i:i+2], color=plt.cm.coolwarm(c[i]))
        ax.scatter(x, y, s=K, color='black')

    # make plot prettier
    ax.set_title(f'd={d[ind]:.2f}')
    ax.set_xlim(-tl * factor, tl * factor)
    ax.set_ylim(-tl * factor, tl * factor)
    ax.set_aspect('equal')
    ax.add_patch(plt.Rectangle((-tl * factor, -tl * factor), 2 * tl * factor, tl * factor, facecolor='gray', linewidth=0))
    ax.set_xlim(-tl * factor, tl * factor)
    ax.set_ylim(-tl * factor * 0.1, tl * factor)

fig.subplots_adjust(top=0.8, wspace=0.2, hspace=0.25)
fig.suptitle(f'$N={len(C)}$,\n$K={arr1D_to_str(K)}$,\n$C={arr1D_to_str(C)}$,\n$L={arr1D_to_str(L)}$')
plt.show()

In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Assuming 'result' dictionary is already defined with necessary data
L = result['L']
K = result['K']
C = result['C']
d = result['d']
N = result['N']

K = np.append(K, 0)

margin = 0.2
factor = margin + 1.0
tl = np.sum(L)

fig, ax = plt.subplots(figsize=(8,5))

fig.suptitle(f'$N={len(C)}$,\n$K={arr1D_to_str(K)}$,\n$C={arr1D_to_str(C)}$,\n$L={arr1D_to_str(L)}$')
ax.set_aspect('equal')
fig.subplots_adjust(top=0.8)

max_abs_str = max([max([max(np.abs(elem2[N+1:])) for elem2 in elem]) for elem in result['q_sol'] if elem != []])
smp = plt.cm.ScalarMappable(cmap=plt.cm.coolwarm, norm=None)
smp.set_clim(vmin=-max_abs_str, vmax=max_abs_str)
cb = fig.colorbar(smp, ax=ax, label='strain')

def animate(i):
    ax.clear()
    for q_sol_temp in result['q_sol'][i]: # iterate over solutions
        angles = q_sol_temp[1:N+1]
        strains = np.array(q_sol_temp[N+1:])
        c = (strains + max_abs_str) / (2 * max_abs_str)
        new_L = L * np.exp(strains)
        x = np.zeros(N + 1)
        y = np.zeros(N + 1)
        x[1:] = np.cumsum(new_L * np.sin(angles))
        y[1:] = np.cumsum(new_L * np.cos(angles))
        for j in range(len(x) - 1):
            ax.plot(x[j:j+2], y[j:j+2], color=plt.cm.coolwarm(c[j]))
        ax.scatter(x, y, s=K*10, color='black')

        ax.add_patch(plt.Rectangle((-tl * factor, -tl * factor), 2 * tl * factor, tl * factor, facecolor='gray', linewidth=0))

        ax.set_title(f'd={d[i]:.2f}')
        ax.set_xlim(-tl * factor, tl * factor)
        ax.set_ylim(-tl * factor*0.1, tl * factor)




ani = animation.FuncAnimation(fig, animate, frames=len(result['q_sol']), repeat=False)
print('saving the animation...')
ani.save('beam_deformation.gif', writer='pillow', fps=50)
print('done saving the animation')
print('showing the animation...')
plt.show()
print('done showing the animation')

# Fix some things about dataset

In [ ]:
import pickle
import torch
import torch.nn.functional as F

In [ ]:
with open('../data/BucklingBeams_data.pkl', 'rb') as f:
    data = pickle.load(f)

for graph in data['data_tr'][:5]:
    print(graph)

In [ ]:
for key in ['data_tr', 'data_te']:
    for i, graph in enumerate(data[key]):
        # convert to float32 instead of float64
        for key, value in graph:
            if value.dtype == torch.float64:
                graph[key] = value.type(torch.float32)
                # print(f"{key}: {value.shape}")


        N = graph.N[0,0].item()  # nr of segments (nr of nodes = N+1)

        # remove useless trailing dimensions
        graph.N = graph.N.reshape(1,)
        graph.d = graph.d.reshape(1, -1)

        # reshape to shape [n+1, dim, T, n_sols]
        graph.pos = torch.transpose(graph.pos, 1, 2)

        # check if x-coordinates of tip have different sign for different time steps
        x_coords_tip = graph.pos[-1, 0, :, 0]  # shape [T,]
        x_coords_tip2 = x_coords_tip[torch.abs(x_coords_tip) > 1e-5] # remove small values
        if torch.any(torch.sign(x_coords_tip2) == 1) and torch.any(torch.sign(x_coords_tip2) == -1):
            print(i)
            print("Warning: x-coordinates of tip have different sign over time steps!")

            sng_tip_end = torch.sign(x_coords_tip[-1])

            # flip x-coordinates of tip if they have different sign than the last time step
            # this is done to ensure that the tip always points in the same direction
            bools = torch.sign(x_coords_tip) != sng_tip_end
            graph.pos[:, 0, bools, :] *= -1


        # add node type as one-hot encoding
        node_type = torch.zeros(N+1, dtype=torch.long)
        node_type[0] = 1  # start node
        node_type[-1] = 2  # end node
        node_type = F.one_hot(node_type, num_classes=3).float()
        graph.node_attr = torch.cat((node_type, graph.node_attr), dim=1)

        graph.lamb = graph.lamb[..., 0, 0]  # remove last two dimensions, keep only one solution (lambda is the same for both solutions anyway)


In [ ]:
data['data_tr'][0]

In [ ]:
with open('../data/BucklingBeams_data_processed.pkl', 'wb') as f:
    pickle.dump(data, f)